In [1]:
import pickle
import pandas as pd

pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)

with open('data.pkl','rb') as file:
    train_data=pickle.load(file)    #pickle方式读取速度会更快
train_data    #把刚才处理的数据给打印出来了！

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,Customer Country,Customer Email,Customer Fname,Customer Id,Customer Lname,Customer Password,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Id,Department Name,Latitude,Longitude,Market,Order City,Order Country,Order Customer Id,order date (DateOrders),Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode,Customer Full Name,order_year,order_month,order_week_day,order_hour,order_month_year
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Cally,20755,Holloway,XXXXXXXXX,Consumer,PR,5365 Noble Nectar Island,725.0,2,Fitness,18.251453,-66.037056,Pacific Asia,Bekasi,Indonesia,20755,2018-01-31 22:56:00,77202,1360,13.110000,0.04,180517,327.750000,0.29,1,327.750000,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,2/3/2018 22:56,Standard Class,CallyHolloway,2018,1,2,22,2018-01
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Irene,19492,Luna,XXXXXXXXX,Consumer,PR,2679 Rustic Loop,725.0,2,Fitness,18.279451,-66.037064,Pacific Asia,Bikaner,India,19492,2018-01-13 12:27:00,75939,1360,16.389999,0.05,179254,327.750000,-0.80,1,327.750000,311.359985,-249.089996,South Asia,Rajastán,PENDING,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/18/2018 12:27,Standard Class,IreneLuna,2018,1,5,12,2018-01
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,EE. UU.,XXXXXXXXX,Gillian,19491,Maldonado,XXXXXXXXX,Consumer,CA,8510 Round Bear Gate,95125.0,2,Fitness,37.292233,-121.881279,Pacific Asia,Bikaner,India,19491,2018-01-13 12:06:00,75938,1360,18.030001,0.06,179253,327.750000,-0.80,1,327.750000,309.720001,-247.779999,South Asia,Rajastán,CLOSED,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/17/2018 12:06,Standard Class,GillianMaldonado,2018,1,5,12,2018-01
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,EE. UU.,XXXXXXXXX,Tana,19490,Tate,XXXXXXXXX,Home Office,CA,3200 Amber Bend,90027.0,2,Fitness,34.125946,-118.291016,Pacific Asia,Townsville,Australia,19490,2018-01-13 11:45:00,75937,1360,22.940001,0.07,179252,327.750000,0.08,1,327.750000,304.809998,22.860001,Oceania,Queensland,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/16/2018 11:45,Standard Class,TanaTate,2018,1,5,11,2018-01
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Orli,19489,Hendricks,XXXXXXXXX,Corporate,PR,8671 Iron Anchor Corners,725.0,2,Fitness,18.253769,-66.037048,Pacific Asia,Townsville,Australia,19489,2018-01-13 11:24:00,75936,1360,29.500000,0.09,179251,327.750000,0.45,1,327.750000,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.750000,0,1/15/2018 11:24,Standard Class,OrliHendricks,2018,1,5,11,2018-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,4,4,40.000000,399.980011,Shipping on time,0,45,Fishing,Brooklyn,EE. UU.,XXXXXXXXX,Maria,1005,Peterson,XXXXXXXXX,Home Office,NY,1322 Broad Glade,11207.0,7,Fan Shop,40.640930,-73.942711,Pa

In [2]:
import numpy as np
train_data['fraud']=np.where( train_data['Order Status']=='SUSPECTED_FRAUD',1,0)
print("train_data中fruad总数",train_data['fraud'].sum())
train_data['fraud']#18万个订单中 有4062个订单是欺诈行为

train_data中fruad总数 4062


0         0
1         0
2         0
3         0
4         0
         ..
180514    0
180515    0
180516    0
180517    0
180518    0
Name: fraud, Length: 180519, dtype: int64

In [3]:
train_data.columns

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)', 'Benefit per order', 'Sales per customer', 'Delivery Status', 'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City', 'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Id', 'Customer Lname', 'Customer Password', 'Customer Segment', 'Customer State', 'Customer Street', 'Customer Zipcode', 'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market', 'Order City', 'Order Country', 'Order Customer Id', 'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id', 'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id', 'Order Item Product Price', 'Order Item Profit Ratio', 'Order Item Quantity', 'Sales', 'Order Item Total', 'Order Profit Per Order', 'Order Region', 'Order State', 'Order Status', 'Order Zipcode', 'Product Card Id', 'Product Category Id', 'Product Description', 'Product Image', 'Product Name', 'Product Price', 'Product Status', 'shipping date (Da

In [4]:
#看一下延迟发货的数据情况
train_data['Delivery Status'].value_counts()#可以看到延迟发货还是很常见的

Late delivery        98977
Advance shipping     41592
Shipping on time     32196
Shipping canceled     7754
Name: Delivery Status, dtype: int64

In [5]:
train_data['Late delivery']=np.where( train_data['Delivery Status']=='Late delivery',1,0)#延迟发货是0，不是延迟发货就是1
train_data['Late delivery'] #这两个字段就搭建完了

0         0
1         1
2         0
3         0
4         0
         ..
180514    0
180515    1
180516    1
180517    0
180518    0
Name: Late delivery, Length: 180519, dtype: int64

In [6]:
#标签fraud和Late delivery搭建完了，把Order Status 和Delivery Status  这两个给删除了 ，
#因为标签就是从他们身上来的，不删除他们训练的模型会是全满分，标签泄漏   还有那个Late_delivery_risk
train_data.drop(['Order Status','Delivery Status','Late_delivery_risk'],axis=1,inplace=True)

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 58 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Type                           180519 non-null  object        
 1   Days for shipping (real)       180519 non-null  int64         
 2   Days for shipment (scheduled)  180519 non-null  int64         
 3   Benefit per order              180519 non-null  float64       
 4   Sales per customer             180519 non-null  float64       
 5   Category Id                    180519 non-null  int64         
 6   Category Name                  180519 non-null  object        
 7   Customer City                  180519 non-null  object        
 8   Customer Country               180519 non-null  object        
 9   Customer Email                 180519 non-null  object        
 10  Customer Fname                 180519 non-null  object        
 11  

In [8]:
#把数据里面类型是object找出来
categeorical_cols=train_data.select_dtypes(include='object').columns
categeorical_cols

Index(['Type', 'Category Name', 'Customer City', 'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Lname', 'Customer Password', 'Customer Segment', 'Customer State', 'Customer Street', 'Department Name', 'Market', 'Order City', 'Order Country', 'Order Region', 'Order State', 'Product Image', 'Product Name', 'shipping date (DateOrders)', 'Shipping Mode', 'Customer Full Name'], dtype='object')

In [9]:
#可以看到数据里面'Customer Password'和Customer Emailz这两列全是XXXXXXXX，就删掉吧！ 可以数据先探索试试
print(train_data['Customer Password'].value_counts())
print(train_data['Customer Email'].value_counts()) #核对完毕 可以删除了

XXXXXXXXX    180519
Name: Customer Password, dtype: int64
XXXXXXXXX    180519
Name: Customer Email, dtype: int64


In [10]:
print("删除这几个特征之前数据特征的个数:{}".format( len(train_data.columns )       )        )
#同理可以探索一下其他的数据，这次把一些其他的没有用的给删除 经过探索有如下几个
train_data.drop(['Customer Email','Customer Password','Product Description','Product Image','Customer Fname','Customer Lname','Product Status'],axis=1,inplace=True)
print("删除后数据特征的个数{}".format(len(train_data.columns)  ))
#可以看到 删除了7个杂碎

删除这几个特征之前数据特征的个数:58
删除后数据特征的个数51


In [11]:
#除此之外可以通过相关性的热力图 把相关性很高的 给删除一部分
train_data.drop(['Order Customer Id','Order Item Cardprod Id','Order Item Id','Sales per customer','Order Item Total',
                 'Order Profit Per Order','Product Card Id',
                 'Product Category Id',  'Product Price','order date (DateOrders)','order_month_year'],axis=1,inplace=True)
print("再删除几个之后的个数{}".format(len(train_data.columns)  ))

再删除几个之后的个数40


In [12]:
pd.set_option('display.max_rows', 10)
train_data['Order Zipcode'].value_counts()#可以看到它缺失值太多了，也需要把它给删除

10035.0    648
10009.0    550
10024.0    541
94122.0    526
10011.0    463
          ... 
59801.0      1
60477.0      1
31088.0      1
61832.0      1
30188.0      1
Name: Order Zipcode, Length: 609, dtype: int64

In [13]:
train_data.drop(['Order Zipcode','Customer Street','shipping date (DateOrders)','Latitude','Longitude'],axis=1,inplace=True)
print("再删除几个之后的个数{}".format(len(train_data.columns)  ))

再删除几个之后的个数35


In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 35 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Category Id                    180519 non-null  int64  
 5   Category Name                  180519 non-null  object 
 6   Customer City                  180519 non-null  object 
 7   Customer Country               180519 non-null  object 
 8   Customer Id                    180519 non-null  int64  
 9   Customer Segment               180519 non-null  object 
 10  Customer State                 180519 non-null  object 
 11  Customer Zipcode               180519 non-null  float64
 12  Department Id                 

In [15]:
#查看分类类型
categorical_clos=train_data.select_dtypes(include='object').columns
categorical_clos        #categeorical_cols

Index(['Type', 'Category Name', 'Customer City', 'Customer Country', 'Customer Segment', 'Customer State', 'Department Name', 'Market', 'Order City', 'Order Country', 'Order Region', 'Order State', 'Product Name', 'Shipping Mode', 'Customer Full Name'], dtype='object')

In [16]:
train_data[categorical_clos]

,Type,Category Name,Customer City,Customer Country,Customer Segment,Customer State,Department Name,Market,Order City,Order Country,Order Region,Order State,Product Name,Shipping Mode,Customer Full Name
0,DEBIT,Sporting Goods,Caguas,Puerto Rico,Consumer,PR,Fitness,Pacific Asia,Bekasi,Indonesia,Southeast Asia,Java Occidental,Smart watch,Standard Class,CallyHolloway
1,TRANSFER,Sporting Goods,Caguas,Puerto Rico,Consumer,PR,Fitness,Pacific Asia,Bikaner,India,South Asia,Rajastán,Smart watch,Standard Class,IreneLuna
2,CASH,Sporting Goods,San Jose,EE. UU.,Consumer,CA,Fitness,Pacific Asia,Bikaner,India,South Asia,Rajastán,Smart watch,Standard Class,GillianMaldonado
3,DEBIT,Sporting Goods,Los Angeles,EE. UU.,Home Office,CA,Fitness,Pacific Asia,Townsville,Australia,Oceania,Queensland,Smart watch,Standard Class,TanaTate
4,PAYMENT,Sporting Goods,Caguas,Puerto Rico,Corporate,PR,Fitness,Pacific Asia,Townsville,Australia,Oceania,Queensland,Smart watch,Standard Class,OrliHendricks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,Fishing,Brooklyn,EE. UU.,Home Office,NY,Fan Shop,Pacific Asia,Shanghái,China,Eastern Asia,Shanghái,Field & Stream Sportsman 16 Gun Fire Safe,Standard Class,MariaPeterson
180515,DEBIT,Fishing,Bakersfield,EE. UU.,Corporate,CA,Fan Shop,Pacific Asia,Hirakata,Japón,Eastern Asia,Osaka,Field & Stream Sportsman 16 Gun Fire Safe,Second Class,RonaldClark
180516,TRANSFER,Fishing,Bristol,EE. UU.,Corporate,CT,Fan Shop,Pacific Asia,Adelaide,Australia,Oceania,Australia del Sur,Field & Stream Sportsman 16 Gun Fire Safe,Standard Class,JohnSmith
180517,PAYMENT,Fishing,Caguas,Puerto Rico,Consumer,PR,Fan Shop,Pacific Asia,Adelaide,Australia,Oceania,Australia del Sur,Field & Stream Sportsman 16 Gun Fire Safe,Standard Class,MarySmith


In [17]:
train_data['Customer Full Name'].isnull().sum() #有8个缺失值

8

In [18]:
#缺失值直接给去掉
train_data=train_data.dropna(subset=['Customer Full Name'])

In [19]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for cat in categorical_clos:
    train_data[cat]=le.fit_transform(train_data[cat])
train_data[categorical_clos]    
#这样就做LabelEncoder了

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Type,Category Name,Customer City,Customer Country,Customer Segment,Customer State,Department Name,Market,Order City,Order Country,Order Region,Order State,Product Name,Shipping Mode,Customer Full Name
0,1,40,66,1,0,36,4,3,331,70,15,475,78,3,1875
1,3,40,66,1,0,36,4,3,391,69,13,841,78,3,5374
2,0,40,452,0,0,5,4,3,391,69,13,841,78,3,4426
3,1,40,285,0,2,5,4,3,3226,8,11,835,78,3,12922
4,2,40,66,1,1,36,4,3,3226,8,11,835,78,3,10632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,0,18,59,0,2,31,3,3,2922,31,7,913,24,3,8421
180515,1,18,26,0,1,5,3,3,1362,77,7,770,24,2,11672
180516,3,18,55,0,1,7,3,3,25,8,11,88,24,3,6296
180517,2,18,66,1,0,36,3,3,25,8,11,88,24,3,9379


In [20]:
#得到所有的数值类型
numerical_columns=train_data.columns.tolist()
for x in categorical_clos.tolist():
    numerical_columns.remove(x)
numerical_columns    #这些类型就是数值类型，有了这些之后可以后续为他处理

['Days for shipping (real)',
 'Days for shipment (scheduled)',
 'Benefit per order',
 'Category Id',
 'Customer Id',
 'Customer Zipcode',
 'Department Id',
 'Order Id',
 'Order Item Discount',
 'Order Item Discount Rate',
 'Order Item Product Price',
 'Order Item Profit Ratio',
 'Order Item Quantity',
 'Sales',
 'order_year',
 'order_month',
 'order_week_day',
 'order_hour',
 'fraud',
 'Late delivery']

In [21]:
x_fraud=train_data.loc[:,train_data.columns!='fraud']
y_fraud=train_data['fraud']
print(y_fraud)
print("-"*100)
x_fraud

0         0
1         0
2         0
3         0
4         0
         ..
180514    0
180515    0
180516    0
180517    0
180518    0
Name: fraud, Length: 180511, dtype: int64
----------------------------------------------------------------------------------------------------


,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Category Id,Category Name,Customer City,Customer Country,Customer Id,Customer Segment,Customer State,Customer Zipcode,Department Id,Department Name,Market,Order City,Order Country,Order Id,Order Item Discount,Order Item Discount Rate,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Region,Order State,Product Name,Shipping Mode,Customer Full Name,order_year,order_month,order_week_day,order_hour,Late delivery
0,1,3,4,91.250000,73,40,66,1,20755,0,36,725.0,2,4,3,331,70,77202,13.110000,0.04,327.750000,0.29,1,327.750000,15,475,78,3,1875,2018,1,2,22,0
1,3,5,4,-249.089996,73,40,66,1,19492,0,36,725.0,2,4,3,391,69,75939,16.389999,0.05,327.750000,-0.80,1,327.750000,13,841,78,3,5374,2018,1,5,12,1
2,0,4,4,-247.779999,73,40,452,0,19491,0,5,95125.0,2,4,3,391,69,75938,18.030001,0.06,327.750000,-0.80,1,327.750000,13,841,78,3,4426,2018,1,5,12,0
3,1,3,4,22.860001,73,40,285,0,19490,2,5,90027.0,2,4,3,3226,8,75937,22.940001,0.07,327.750000,0.08,1,327.750000,11,835,78,3,12922,2018,1,5,11,0
4,2,2,4,134.210007,73,40,66,1,19489,1,36,725.0,2,4,3,3226,8,75936,29.500000,0.09,327.750000,0.45,1,327.750000,11,835,78,3,10632,2018,1,5,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,0,4,4,40.000000,45,18,59,0,1005,2,31,11207.0,7,3,3,2922,31,26043,0.000000,0.00,399.980011,0.10,1,399.980011,7,913,24,3,8421,2016,1,5,3,0
180515,1,3,2,-613.770019,45,18,26,0,9141,1,5,93304.0,7,3,3,1362,77,26037,4.000000,0.01,399.980011,-1.55,1,399.980011,7,770,24,2,11672,2016,1,5,1,1
180516,3,5,4,141.110001,45,18,55,0,291,1,7,6010.0,7,3,3,25,8,26024,8.000000,0.02,399.980011,0.36,1,399.980011,11,88,24,3,6296,2016,1,4,21,1
180517,2,3,4,186.229996,45,18,66,1,2813,0,36,725.0,7,3,3,25,8,26022,12.000000,0.03,399.980011,0.48,1,399.980011,11,88,24,3,9379,2016,1,4,20,0


In [22]:
x_late=train_data.loc[:,train_data.columns!='Late delivery']
y_late=train_data['Late delivery']
print(y_fraud)
print("-"*100)
x_fraud

0         0
1         0
2         0
3         0
4         0
         ..
180514    0
180515    0
180516    0
180517    0
180518    0
Name: fraud, Length: 180511, dtype: int64
----------------------------------------------------------------------------------------------------


,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Category Id,Category Name,Customer City,Customer Country,Customer Id,Customer Segment,Customer State,Customer Zipcode,Department Id,Department Name,Market,Order City,Order Country,Order Id,Order Item Discount,Order Item Discount Rate,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Region,Order State,Product Name,Shipping Mode,Customer Full Name,order_year,order_month,order_week_day,order_hour,Late delivery
0,1,3,4,91.250000,73,40,66,1,20755,0,36,725.0,2,4,3,331,70,77202,13.110000,0.04,327.750000,0.29,1,327.750000,15,475,78,3,1875,2018,1,2,22,0
1,3,5,4,-249.089996,73,40,66,1,19492,0,36,725.0,2,4,3,391,69,75939,16.389999,0.05,327.750000,-0.80,1,327.750000,13,841,78,3,5374,2018,1,5,12,1
2,0,4,4,-247.779999,73,40,452,0,19491,0,5,95125.0,2,4,3,391,69,75938,18.030001,0.06,327.750000,-0.80,1,327.750000,13,841,78,3,4426,2018,1,5,12,0
3,1,3,4,22.860001,73,40,285,0,19490,2,5,90027.0,2,4,3,3226,8,75937,22.940001,0.07,327.750000,0.08,1,327.750000,11,835,78,3,12922,2018,1,5,11,0
4,2,2,4,134.210007,73,40,66,1,19489,1,36,725.0,2,4,3,3226,8,75936,29.500000,0.09,327.750000,0.45,1,327.750000,11,835,78,3,10632,2018,1,5,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,0,4,4,40.000000,45,18,59,0,1005,2,31,11207.0,7,3,3,2922,31,26043,0.000000,0.00,399.980011,0.10,1,399.980011,7,913,24,3,8421,2016,1,5,3,0
180515,1,3,2,-613.770019,45,18,26,0,9141,1,5,93304.0,7,3,3,1362,77,26037,4.000000,0.01,399.980011,-1.55,1,399.980011,7,770,24,2,11672,2016,1,5,1,1
180516,3,5,4,141.110001,45,18,55,0,291,1,7,6010.0,7,3,3,25,8,26024,8.000000,0.02,399.980011,0.36,1,399.980011,11,88,24,3,6296,2016,1,4,21,1
180517,2,3,4,186.229996,45,18,66,1,2813,0,36,725.0,7,3,3,25,8,26022,12.000000,0.03,399.980011,0.48,1,399.980011,11,88,24,3,9379,2016,1,4,20,0


In [23]:
#数据集切分
from sklearn.model_selection import train_test_split
x_fraud_train,x_fraud_test,y_fraud_train,y_fraud_test=train_test_split(x_fraud,y_fraud,test_size=0.2)
x_late_train,x_late_test,y_late_train,y_late_test=train_test_split(x_late,y_late,test_size=0.2)

In [24]:
#数据规范化一下
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_fraud_train=sc.fit_transform(x_fraud_train)
x_fraud_test=sc.transform(x_fraud_test)

x_late_train=sc.fit_transform(x_late_train)
x_late_test=sc.transform(x_late_test)

In [25]:
from sklearn.metrics import accuracy_score,recall_score,roc_auc_score,confusion_matrix,f1_score
#定义一个模型函数
def model_stats(model,x_train,x_test,y_train,y_test,name='Fraud'):
    model=model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    accuracy=accuracy_score(y_pred,y_test)
    recall=recall_score(y_pred,y_test)
    auc=roc_auc_score(y_pred,y_test)   
    f1=f1_score(y_pred,y_test)   
    confusion=confusion_matrix(y_pred,y_test)
    print("预测:{}".format(name),'-'*80)
    print("使用的模型：{}".format(model))
    print("准确率：{}".format(accuracy))
    print("召回率：{}".format(recall))
    print("Auc：{}".format(auc))
    print("F1-score：{}".format(f1))
    print("混淆矩阵：\n{}".format(confusion))
    print("预测:{}".format(name),'-'*80)

In [26]:
#逻辑回归模型
from sklearn.linear_model import LogisticRegression
model_fraud=LogisticRegression()
model_late=LogisticRegression()

model_stats(model_fraud,x_fraud_train,x_fraud_test,y_fraud_train,y_fraud_test,'fraud')
model_stats(model_late,x_late_train,x_late_test,y_late_train,y_late_test,'late')

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


预测:fraud --------------------------------------------------------------------------------
使用的模型：LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
准确率：0.9793645957399663
召回率：0.6146496815286624
Auc：0.7986070783233576
F1-score：0.34129089301503096
混淆矩阵：
[[35165   624]
 [  121   193]]
预测:fraud --------------------------------------------------------------------------------
预测:late --------------------------------------------------------------------------------
使用的模型：LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='